In [ ]:
import pandas as pd

In [ ]:
label = pd.read_json(r"C:\Users\phani\Downloads\imagenet_class_index.json")
label = label.T[[1]]

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import glob

In [ ]:
model = VGG16(weights="imagenet") # 1000 class classification

In [ ]:
input_img= image.load_img(r"C:\Users\phani\OneDrive\Trainings\IP Training AI\AI and Deep Learning using TensorFLow\Module 5\In-Class Demo\ImagesToTest\big_Shadow_1yr_old.jpg" ,target_size=(224,224))
input_img = np.array(input_img).reshape(-1,224,224,3)
input_img = preprocess_input(input_img)

In [ ]:
preds = model.predict(input_img).flatten()
output = pd.DataFrame({"id":range(len(preds)), "prob":preds})
output.join(label).sort_values("prob", ascending=False)

,id,prob,1
250,250,5.650504e-01,Siberian_husky
248,248,3.791240e-01,Eskimo_dog
249,249,5.144948e-02,malamute
537,537,1.707264e-03,dogsled
235,235,7.207285e-04,German_shepherd
...,...,...,...
92,92,3.636850e-10,bee_eater
37,37,3.218674e-10,box_turtle
91,91,2.804102e-10,coucal
45,45,1.963449e-10,Gila_monster


#### Finetuning VGG16 with custom data

In [ ]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
newModel = Sequential()#empty model
for layer in model.layers[:-1]:
    layer.trainable = False
    newModel.add(layer)

newModel.add(Dense(1, "sigmoid"))

In [ ]:
newModel.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [ ]:
newModel.compile(optimizer = "adam", loss="binary_crossentropy",metrics=["accuracy"])

In [ ]:
cars = "ImagesToTest/ImagesToTrain/Cars/*"
planes = "ImagesToTest/ImagesToTrain/Planes/*"


In [ ]:
x_train = list()
for i in glob.glob(cars):
    input_img = image.load_img(i, target_size=(224,224))
    input_img = np.array(input_img)
    x_train.append(input_img)

for i in glob.glob(planes):
    input_img = image.load_img(i, target_size=(224,224))
    input_img = np.array(input_img)
    x_train.append(input_img)

In [ ]:
y_train = [0]*50 + [1]*50

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)

In [ ]:
x_train.shape

(100, 224, 224, 3)

In [ ]:
y_train.shape

(100,)

In [ ]:
x_train = preprocess_input(x_train)

In [ ]:
newModel.fit(x=x_train, y=y_train,epochs=20,batch_size=30, validation_split=0.3,shuffle=True)

Epoch 1/20
3/3 [==============================] - 9s 3s/step - loss: 0.5202 - accuracy: 0.7000 - val_loss: 0.4738 - val_accuracy: 0.6667
Epoch 2/20
3/3 [==============================] - 9s 3s/step - loss: 0.0469 - accuracy: 1.0000 - val_loss: 0.2648 - val_accuracy: 0.8667
Epoch 3/20
3/3 [==============================] - 10s 3s/step - loss: 0.0106 - accuracy: 1.0000 - val_loss: 0.1737 - val_accuracy: 0.9000
Epoch 4/20
3/3 [==============================] - 9s 3s/step - loss: 0.0037 - accuracy: 1.0000 - val_loss: 0.1322 - val_accuracy: 0.9333
Epoch 5/20
3/3 [==============================] - 9s 3s/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.1088 - val_accuracy: 0.9667
Epoch 6/20
3/3 [==============================] - 9s 3s/step - loss: 9.4575e-04 - accuracy: 1.0000 - val_loss: 0.0938 - val_accuracy: 0.9667
Epoch 7/20
3/3 [==============================] - 9s 3s/step - loss: 5.6457e-04 - accuracy: 1.0000 - val_loss: 0.0840 - val_accuracy: 0.9667
Epoch 8/20
3/3 [================